In [36]:
# Imports
import numpy as np
import math
import pandas
import bokeh
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.layouts import column
from sklearn import datasets, metrics
from sklearn.linear_model import Ridge
import numpy.linalg as linalg

In [37]:
# Data importing and organizing
X,Y = datasets.make_regression(n_samples=50, n_features=10,n_targets=1,noise=2)
X = X.T
ones = np.ones(len(X[0]))
X = np.append(X,[ones],axis=0)
X = X.T
X_test = X[int(len(X)/2):]
Y_test = Y[int(len(Y)/2):]
X = X[:int(len(X)/2)]
Y = Y[:int(len(Y)/2)]

In [38]:
# Alghoritm definition  F = V D U^T
def msum(a,b):
    res = []
    for i in range(len(a)):
        res_sub = []
        for j in range(len(a[i])):
            res_sub.append(a[i][j]+b[i][j])
        res.append(res_sub)
    return np.asarray(res)

def msub(a,b):
    res = []
    for i in range(len(a)):
        res_sub = []
        for j in range(len(a[i])):
            res_sub.append(a[i][j]-b[i][j])
        res.append(res_sub)
    return np.asarray(res)

def mmult(a,b):
    return np.asarray(np.dot(a,b))

def minv(a):
    return np.asarray(linalg.inv(a))

def mI(val,size):
    res = []
    for i in range(size):
        res_sub = []
        for j in range(size):
            res_sub.append(val if i==j else 0)
        res.append(res_sub)
    return np.asarray(res)

def mIs(val,size):
    res = []
    for i in range(size):
        res_sub = []
        for j in range(size):
            res_sub.append((val[i] if i<len(val) else 0) if i==j else 0)
        res.append(res_sub)
    return np.asarray(res)

def ridge(X,y,tau):
    a = mmult(X,X.T)
    a = msum(a,mI(tau,len(a)))
    a = minv(a)
    a = mmult(X.T,a)
    a = mmult(a,y)
    return a

In [39]:
# Calculations
errors = [[x/10 for x in range(0,1001)]]
err_vals = []
err_vecs = []
for i in range(len(errors[0])):
    tau = errors[0][i]
    tcoef = ridge(X,Y,tau)
    err_vals.append(metrics.mean_squared_error(Y_test,[tcoef[1]+x*tcoef[0] for x in X_test[:,0]]))
    err_vecs.append([x for x in tcoef])
errors = np.append(errors,[err_vals],axis=0)

min_id = errors[1].tolist().index(errors[1].min())
coefs = ridge(X,Y,errors[0][min_id])

In [40]:
# Visualizations
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,reset,tap,save,"
p = figure(tools=TOOLS)
p.scatter(X[:,0],Y)
p.scatter(X_test[:,0],Y_test,fill_color=["#%02x%02x%02x" % (255,0,0) for w in Y_test])
p.line(X[:,0],coefs[1]+X[:,0]*coefs[0])

p2 = figure(tools = TOOLS)
p2.line(errors[0],errors[1])
p2.xaxis.axis_label = 'tau'
p2.yaxis.axis_label = 'squared error'

p3 = figure(tools = TOOLS)
err_vecs = np.asarray(err_vecs)

colors = bokeh.palettes.d3['Category20'][20]
for i in range(len(err_vecs[0])):
    p3.line(errors[0],err_vecs[:,i],line_color=colors[i])
p3.xaxis.axis_label = 'tau'
p3.yaxis.axis_label = 'coef value'

output_file("RidgeRegression.html", title="Ridge")
output_notebook()

show(column(p,p2,p3))

Loading BokehJS ...